In [12]:
from sklearn.linear_model import Lasso, Ridge, LinearRegression
from sklearn.ensemble import VotingRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
import pandas as pd
import numpy as np

In [25]:
df_airport = pd.read_csv('Airport Codes Dataset.csv')
print(df_airport.shape)
airport_rename = df_airport.rename(columns={'city':'Ori-city','state':'Ori-state','name':'Ori-airport'})
airport_rename.head()

(365, 4)


,airport_id,Ori-city,Ori-state,Ori-airport
0,10165,Adak Island,AK,Adak
1,10299,Anchorage,AK,Ted Stevens Anchorage International
2,10304,Aniak,AK,Aniak Airport
3,10754,Barrow,AK,Wiley Post/Will Rogers Memorial
4,10551,Bethel,AK,Bethel Airport


In [26]:
df_flight = pd.read_csv('Flight on-time performance.csv')
df_flight.head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,Carrier,OriginAirportID,DestAirportID,CRSDepTime,DepTimeBlk,DepDelay,DepDel15,CRSArrTime,ArrTimeBlk,ArrDelay,ArrDel15,Cancelled,Diverted
0,2011,4,10,6,4,WN,13495,12191,1435,1400-1459,2.0,0.0,1550,1500-1559,-6.0,0.0,0,0
1,2011,4,10,6,4,WN,13495,12191,1330,1300-1359,-4.0,0.0,1445,1400-1459,-12.0,0.0,0,0
2,2011,4,10,6,4,WN,13495,12191,1030,1000-1059,-2.0,0.0,1145,1100-1159,-14.0,0.0,0,0
3,2011,4,10,6,4,WN,13495,12889,1900,1900-1959,0.0,0.0,2055,2000-2059,-6.0,0.0,0,0
4,2011,4,10,6,4,WN,13495,12889,1340,1300-1359,-1.0,0.0,1530,1500-1559,2.0,0.0,0,0


In [28]:
# 결측치확인
df_flight.isna().sum()

Year                  0
Quarter               0
Month                 0
DayofMonth            0
DayOfWeek             0
Carrier               0
OriginAirportID       0
DestAirportID         0
CRSDepTime            0
DepTimeBlk            0
DepDelay           3697
DepDel15           3697
CRSArrTime            0
ArrTimeBlk            0
ArrDelay           4717
ArrDel15           4717
Cancelled             0
Diverted              0
dtype: int64

In [15]:
flight_outer = pd.merge(airport_rename,df_flight,left_on='airport_id',right_on='OriginAirportID')
print(flight_outer.shape)

(504397, 22)


In [16]:
airport_outer = airport.rename(columns={'city':'Dest-city','state':'Dest-state','name':'Dest-airport'})
airport_outer


,airport_id,Dest-city,Dest-state,Dest-airport
0,10165,Adak Island,AK,Adak
1,10299,Anchorage,AK,Ted Stevens Anchorage International
2,10304,Aniak,AK,Aniak Airport
3,10754,Barrow,AK,Wiley Post/Will Rogers Memorial
4,10551,Bethel,AK,Bethel Airport
...,...,...,...,...
360,11233,Cheyenne,WY,Cheyenne Regional/Jerry Olson Field
361,11097,Cody,WY,Yellowstone Regional
362,11865,Gillette,WY,Gillette Campbell County
363,12441,Jackson,WY,Jackson Hole


In [17]:
flight_outer = pd.merge(airport_outer,flight_outer, left_on='airport_id',right_on='OriginAirportID')
flight_outer.shape

(504397, 26)

In [18]:
flight_outer =flight_outer.drop('airport_id_x',axis=1)
flight_outer =flight_outer.drop('airport_id_y',axis=1)
flight_outer

,Dest-city,Dest-state,Dest-airport,Ori-city,Ori-state,Ori-airport,Year,Quarter,Month,DayofMonth,...,CRSDepTime,DepTimeBlk,DepDelay,DepDel15,CRSArrTime,ArrTimeBlk,ArrDelay,ArrDel15,Cancelled,Diverted
0,Adak Island,AK,Adak,Adak Island,AK,Adak,2011,4,10,2,...,1800,1800-1859,25.0,1.0,2143,2100-2159,14.0,0.0,0,0
1,Adak Island,AK,Adak,Adak Island,AK,Adak,2011,4,10,6,...,1800,1800-1859,5.0,0.0,2143,2100-2159,8.0,0.0,0,0
2,Adak Island,AK,Adak,Adak Island,AK,Adak,2011,4,10,9,...,1800,1800-1859,-8.0,0.0,2143,2100-2159,-13.0,0.0,0,0
3,Adak Island,AK,Adak,Adak Island,AK,Adak,2011,4,10,13,...,1800,1800-1859,66.0,1.0,2143,2100-2159,58.0,1.0,0,0
4,Adak Island,AK,Adak,Adak Island,AK,Adak,2011,4,10,16,...,1800,1800-1859,22.0,1.0,2143,2100-2159,21.0,1.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504392,Rock Springs,WY,Rock Springs Sweetwater County,Rock Springs,WY,Rock Springs Sweetwater County,2011,4,10,15,...,2110,2100-2159,-9.0,0.0,2219,2200-2259,-22.0,0.0,0,0
504393,Rock Springs,WY,Rock Springs Sweetwater County,Rock Springs,WY,Rock Springs Sweetwater County,2011,4,10,22,...,2110,2100-2159,-20.0,0.0,2219,2200-2259,-31.0,0.0,0,0
504394,Rock Springs,WY,Rock Springs Sweetwater County,Rock Springs,WY,Rock Springs Sweetwater County,2011,4,10,14,...,2110,2100-2159,-8.0,0.0,2219,2200-2259,-15.0,0.0,0,0
504395,Rock Springs,WY,Rock Springs Sweetwater County,Rock Springs,WY,Rock Springs Sweetwater County,2011,4,10,23,...,2110,2100-2159,-14.0,0.0,2219,2200-2259,-29.0,0.0,0,0


In [20]:
flight_outer = flight_outer[['DayOfWeek', 'Carrier', 'DepTimeBlk', 'DepDelay', 'DepDel15', 'ArrTimeBlk', 'ArrDel15',
'Ori-city', 'Ori-state', 'Ori-airport', 'Dest-city', 'Dest-state', 'Dest-airport', 'ArrDelay']]
flight_outer.head()

,DayOfWeek,Carrier,DepTimeBlk,DepDelay,DepDel15,ArrTimeBlk,ArrDel15,Ori-city,Ori-state,Ori-airport,Dest-city,Dest-state,Dest-airport,ArrDelay
0,7,AS,1800-1859,25.0,1.0,2100-2159,0.0,Adak Island,AK,Adak,Adak Island,AK,Adak,14.0
1,4,AS,1800-1859,5.0,0.0,2100-2159,0.0,Adak Island,AK,Adak,Adak Island,AK,Adak,8.0
2,7,AS,1800-1859,-8.0,0.0,2100-2159,0.0,Adak Island,AK,Adak,Adak Island,AK,Adak,-13.0
3,4,AS,1800-1859,66.0,1.0,2100-2159,1.0,Adak Island,AK,Adak,Adak Island,AK,Adak,58.0
4,7,AS,1800-1859,22.0,1.0,2100-2159,1.0,Adak Island,AK,Adak,Adak Island,AK,Adak,21.0


In [22]:
flight_outer = flight_outer.dropna()
flight_outer

,DayOfWeek,Carrier,DepTimeBlk,DepDelay,DepDel15,ArrTimeBlk,ArrDel15,Ori-city,Ori-state,Ori-airport,Dest-city,Dest-state,Dest-airport,ArrDelay
0,7,AS,1800-1859,25.0,1.0,2100-2159,0.0,Adak Island,AK,Adak,Adak Island,AK,Adak,14.0
1,4,AS,1800-1859,5.0,0.0,2100-2159,0.0,Adak Island,AK,Adak,Adak Island,AK,Adak,8.0
2,7,AS,1800-1859,-8.0,0.0,2100-2159,0.0,Adak Island,AK,Adak,Adak Island,AK,Adak,-13.0
3,4,AS,1800-1859,66.0,1.0,2100-2159,1.0,Adak Island,AK,Adak,Adak Island,AK,Adak,58.0
4,7,AS,1800-1859,22.0,1.0,2100-2159,1.0,Adak Island,AK,Adak,Adak Island,AK,Adak,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504392,6,OO,2100-2159,-9.0,0.0,2200-2259,0.0,Rock Springs,WY,Rock Springs Sweetwater County,Rock Springs,WY,Rock Springs Sweetwater County,-22.0
504393,6,OO,2100-2159,-20.0,0.0,2200-2259,0.0,Rock Springs,WY,Rock Springs Sweetwater County,Rock Springs,WY,Rock Springs Sweetwater County,-31.0
504394,5,OO,2100-2159,-8.0,0.0,2200-2259,0.0,Rock Springs,WY,Rock Springs Sweetwater County,Rock Springs,WY,Rock Springs Sweetwater County,-15.0
504395,7,OO,2100-2159,-14.0,0.0,2200-2259,0.0,Rock Springs,WY,Rock Springs Sweetwater County,Rock Springs,WY,Rock Springs Sweetwater County,-29.0


In [37]:
y_hat = gb_c.predict(X_test)
print(f'정확도: {metrics.accuracy_score(y_test,y_hat):.3f}')
auc = metrics.roc_auc_score(y_test,gb_c.predict_proba(X_test)[:,1])
print(f'auc: {auc:.3f}')

정확도: 0.942
auc: 0.987


# 4. GradiendtBoostringRegressor (부스팅 회귀)